# Bangsamoro Earthquake Data Visualization
## Links
1. https://towardsdatascience.com/how-to-create-an-interactive-geographic-map-using-python-and-bokeh-12981ca0b567

## Instructions(Local)
0. Make sure packages required are installed : `pip install -r requirements.txt`
1. Running this notebook is not needed. We can go directly to command line and run `bokeh serve path/to/this/ipynb --show`
2. Open localhost:5006/earthquake on your browser or whatever link is provided after step 1
3. Wait for a while as bokeh is quite slow(possibly depend on your machine)

## Todo
1. Include non BARMM Geodata as the visualization has a lot of white space, but gray out this non BARMM as this is not part of the project
2. Fix hover, it is not working when changing to medium risk earthquake data
3. Deploy to heroku
4. See if PWA/Offline App possible

In [ ]:
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import shapely.geometry as gm
import json

# Data Processing

## Risk Data

In [ ]:
highrisk = pd.read_csv('data/high_risk0.csv')
medrisk = pd.read_csv('data/midrisk.csv')
medrisk_sea = pd.read_csv('data/medium.csv')

In [ ]:
highrisk = gpd.GeoDataFrame(highrisk.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(highrisk.longitude, highrisk.latitude)])
medrisk = gpd.GeoDataFrame(medrisk.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(medrisk.longitude, medrisk.latitude)])
medrisk_sea = gpd.GeoDataFrame(medrisk_sea.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(medrisk_sea.longitude, medrisk_sea.latitude)])

## Geo Data

In [ ]:
bm = gpd.read_file('mapdata/AdministrativeBoundariesBARMMMunicipalities20190206PSA2016.shp')

In [ ]:
bm = bm.to_crs(epsg=4236)

In [ ]:
bm['geometry'] = [geometry for geometry in bm['geometry'].simplify(0.001).translate(xoff=0.0075,yoff=-0.0019) ]

## Population Data

In [ ]:
population = pd.read_csv('mapdata/municipal_population.csv')

## Combine

In [ ]:
lenbm = len(bm)
lenhr = len(highrisk)
lenmdr = len(medrisk)#combine all medrisk  /
lenmdr2 = len(medrisk_sea)
bm_range = range(lenbm)
highrisk_range = range(lenhr)
medrisk_range = range(lenmdr)
medrisksea_range = range(lenmdr2)

In [ ]:
bangsamoro_risk = {'high':[],'med':[], 'med_sea':[]}
#max_geo_distance = 180 #assuming that max distance of geoobjects is 180'.''
for x in bm_range:
    municipal_highrisk = 0
    municipal_medrisk = 0
    municipal_medrisksea = 0
    for y in highrisk_range:
        municipal_highrisk += bm.geometry[x].centroid.distance(highrisk.geometry[y])
    for z in medrisk_range:
        municipal_medrisk += bm.geometry[x].centroid.distance(medrisk.geometry[z])
    for v in medrisksea_range:
        municipal_medrisksea += bm.geometry[x].centroid.distance(medrisk_sea.geometry[v])
    bangsamoro_risk['high'].append(1/(municipal_highrisk)**2)
    bangsamoro_risk['med'].append(1/(municipal_medrisk)**2)
    bangsamoro_risk['med_sea'].append(1/(municipal_medrisksea)**2)

In [ ]:
#add normalized to geodata for choropleth
bm['high'] = bangsamoro_risk['high']
bm['high'] = bm['high']/bm['high'].max()
bm['med'] = bangsamoro_risk['med']
bm['med'] = bm['med']/bm['med'].max()
bm['med_sea'] = bangsamoro_risk['med_sea']
bm['med_sea'] = bm['med_sea']/bm['med_sea'].max()

In [ ]:
bm = bm.merge(population[['POPULATION','PSGC_CITY/MUNI']], left_on='Mun_Code', right_on='PSGC_CITY/MUNI').drop('PSGC_CITY/MUNI', axis=1)

## Output

In [ ]:
medrisk_sea.to_file("output/medsea.geojson", driver = "GeoJSON")
medrisk.to_file("output/medrisk.geojson", driver = "GeoJSON")
highrisk.to_file("output/highrisk.geojson", driver = "GeoJSON")
bm.to_file("output/merged_data.geojson", driver = "GeoJSON")

# Visualizations

## Builtin/Matplotlib

In [ ]:
x = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))
#y = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))
#z = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))

In [ ]:
highrisk.plot(ax=x, color='red')

In [ ]:
medrisk.plot(ax=x, color='orange')
medrisk_sea.plot(ax=x, color='green')

In [ ]:
x.get_figure()

## Bokeh

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [ ]:
def update_plot(attr,old,new):
    severity=select.value
    new_data = json_data(severity)
    #input_field
    geosource.geojson = new_data
    hover = HoverTool(tooltips = [('Municipality','@Mun_Name'),(severity+' risk','@'+severity)])
    p = make_plot(severity)
    layout=column(p,widgetbox(select))
    curdoc().clear()
    curdoc().add_root(layout)

In [ ]:
def json_data(severity):
    if severity == 'high':
        return json.dumps(json.loads(bm.drop(['med', 'med_sea'], axis=1).to_json()))
    if severity == 'med':
        return json.dumps(json.loads(bm.drop(['high', 'med_sea'], axis=1).to_json()))
    return json.dumps(json.loads(bm.drop(['high', 'med'],axis=1).to_json()))    

In [ ]:
def make_plot(severity):
    color_mapper = LinearColorMapper(palette=palette,low=bm[severity].min(), high=bm[severity].max())
    #format_tick = NumeralTickFormatter(format='0.0')
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
    p = figure(title='Title', plot_height = 650, plot_width = 850, toolbar_location = None)
    #p = figure()
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.patches('xs', 'ys', source=geosource, fill_color = {'field' : severity, 'transform' : color_mapper}, line_color = 'gray', line_width = 0.25, fill_alpha = 1)
    p.add_layout(color_bar, 'right')
    p.add_tools(hover)
    return p

In [ ]:
severity = 'high'
geosource = GeoJSONDataSource(geojson = json_data(severity))

In [ ]:
palette = brewer['Blues'][8]
palette = palette[::-1]
hover = HoverTool(tooltips = [('Municipality','@Mun_Name'),(severity+' risk','@'+severity),('Population','@POPULATION')])
p = make_plot(severity)
select = Select(title='Select Severity:', value=severity, options=['high','med','med_sea'])
#maybe we could add toggle for epicenters and evacuation centers
select.on_change('value', update_plot)
layout=column(p, widgetbox(select))
curdoc().add_root(layout)

In [ ]:
#output_notebook() #comment out when deploying to heroku

In [ ]:
#p= figure() #comment out when deploying to heroku

In [ ]:
#show(p) #comment out when deploying to heroku

## Plotly

In [ ]:
import json
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
bm = gpd.read_file('output/merged_data.geojson')
with open("output/merged_data.geojson") as geofile:
    j_file = json.load(geofile)
with open('output/highrisk.geojson') as highriskfile:
    h_file = json.load(highriskfile)

In [ ]:
for feature in j_file["features"]:
    feature['id']= feature['properties']['Mun_Code']

In [ ]:
risk='med_sea'
color = {'high':'spectral_r', 'med':'oranges', 'med_sea':'algae'}
risk_translation = {'high':'Above Magnitude 5 Risk', 'med': 'Below Magnitude 5 Risk', 'med_sea': 'Below Sea-Floor Below Magnitude 5 Risk'}

In [ ]:
px.set_mapbox_access_token('pk.eyJ1Ijoia2V2c2VzdHJlbGxhIiwiYSI6ImNrNWlwcWpvZDBoNGEza21zeDc0OWczeDIifQ.-ajWL8TrUDMCN1OzzXTjhg')

In [ ]:
fig = px.choropleth_mapbox(bm[['Mun_Name','Mun_Code','POPULATION','high','med', 'med_sea']], geojson=j_file, locations='Mun_Code',
                           color=risk, color_continuous_scale='spectral_r', range_color=(1,0),
                           mapbox_style="light", zoom=6, center = {"lat": 6.509640, "lon": 121.648446},
                           opacity=0.5,labels={'Mun_Name':'Municipality',risk:'risk', 'POPULATION':'Population'}, hover_data = ['Mun_Name', 'POPULATION'])
fig.add_trace(go.Scattergeo(geojson=h_file))
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
 ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg',
             'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn', 'puor',
             'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy',
             'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar', 'spectral',
             'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose',
             'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'twilight',
             'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd'].

In [ ]:
highrisk['geometry']